<a href="https://colab.research.google.com/drive/1_E7kjUJ6IU_NL2jzGjMIBx2dz4q_nvcz?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to GPU Programming with Numba and CuPy
## Estimating Pi with Monte Carlo Simulation

In this notebook, we'll walk through how we can use packages with friendly `numpy`-style syntax -- `numba` and `cupy` -- to accelerate our embarrasingly parallel code using GPUs. Our goal today will be to a estimate a value -- $\pi$ -- [using Monte Carlo Simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method#Overview) (a classic task for demonstrating HPC speedups and applicable to any numerical estimation task in the social sciences).

First taking a look at the hardware we have available to work with:

In [1]:
!nvidia-smi

Wed Mar 29 15:02:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Serially, we can solve this problem in `numpy` like so (generating 100m coordinates in a unit square and identifying whether they fall in a unit circle or not):

In [2]:
# NumPy Pi Estimation with Monte Carlo Simulation
import numpy as np
import time

t0 = time.time()

n_runs = 10 ** 8

# Simulate Random Coordinates in Unit Square:
ran = np.random.uniform(low=-1, high=1, size=(2, n_runs))

# Identify Random Coordinates that fall within Unit Circle and count them
result = ran[0] ** 2 + ran[1] ** 2 <= 1
n_in_circle = np.sum(result)

# Estimate Pi
print("Pi Estimate: ", 4 * n_in_circle / n_runs)

print("Time Elapsed: ", time.time() - t0)

Pi Estimate:  3.141506
Time Elapsed:  2.7714076042175293


## Vectorization on GPUs with `numba`

In addition to compiling code for CPUs, `numba` also gives us the ability to compile (rudimentary CUDA) code for GPUs via the familiar `vectorize` decorator, as well as a `reduce` decorator that we have not yet seen.

*Note: `numba` has a GPU random number generator and can compile GPU code via JIT (such as pi estimation code), but this is a bit more involved and beyond the scope of the class (it involves more CUDA-style syntax). [See the documentation]((https://numba.readthedocs.io/en/stable/cuda/random.html)) for more detail if you are interested.*

In [3]:
from numba import vectorize, cuda
from numba.core.errors import NumbaPerformanceWarning
import warnings
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

n_runs = 10**8
ran = np.random.uniform(low=-1, high =1, size=(2, n_runs)).astype(np.float32)
x, y = ran[0], ran[1]

@vectorize(['i4(f4, f4)'], target='cuda')
def in_circle(x, y):
  '''
  Vectorized function takes in x, y coordinates (float32, float32) within an
  array and returns a boolean indication of whether these values are (1) or
  are not (0) in the unit circle (cast as int32). 
  
  All computation is done on the GPU.
  '''
  in_circle = x**2 + y**2 <= 1
  return in_circle

@cuda.reduce
def gpu_sum(a, b):
  '''
  Sums values in an array together on the GPU, using numba's built-in
  `reduce` algorithm.
  '''
  return a + b

First, we might try to only perform the mapping operation on the GPU (passing the x and y arrays over to the GPU, computing whether the random coordinates are in the unit circle or not, and then sending values back to the CPU to sum and estimate pi). Note that `numba` accepts `numpy` arrays directly and we do not need to perform any (explicit) conversion:

In [4]:
%%timeit
result = in_circle(x, y) # send numpy arrays x & y to GPU and perform in_circle computation on GPU
4 * np.sum(result) / n_runs # send resulting array back to CPU to compute pi using `numpy`

343 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Can also perform the summation on the GPU...

In [5]:
%%timeit
result = in_circle(x, y)
4 * gpu_sum(result) / n_runs # perform sum on GPU this time and then only two scalar ops on CPU

286 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And store intermediate results on the GPU (`result` is currently being sent back to CPU):

In [7]:
%%timeit
# Slight improvement by keeping intermediate result on GPU, but not to the
# degree of custom CUDA kernel.
in_circle_dev = cuda.device_array(shape=(n_runs,), dtype=np.float32) # allocate spot in memory on GPU for output array to live
in_circle(x, y, out=in_circle_dev) # write results of `in_circle` out to spot in memory allocated to `in_circle_dev`
4 * gpu_sum(in_circle_dev) / n_runs

206 ms ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Note that we see even more improved performance by avoiding passing data between CPU and GPU unless absolutely necessary. This capacity to define custom elementwise mapping functions via `@vectorize` can be particularly useful for accelerating existing `numpy` code with minimal tweaks needed. 

## CuPy

Another useful approach for working with array data on GPUs is to use `CuPy`, which replicates much of the functionality of `numpy`, but on GPUs. You can take a look at [the documentation](https://docs.cupy.dev/en/stable/user_guide/basic.html) for more detail on all of the GPU functions that are available in CuPy. If you need to write a function that is not supported in CuPy, you can either [define a your own GPU kernel](https://docs.cupy.dev/en/stable/user_guide/basic.html) (beyond the scope of the class, as this requires some exposure to underlying CUDA syntax), or use `numba` to compile a kernel for you. Note that [CuPy arrays are also communicable via mpi4py](https://docs.cupy.dev/en/stable/user_guide/interoperability.html#mpi4py) in a distributed GPU context.

To estimate $\pi$ for instance, we can generate random numbers on our GPU in exactly the same way we did in `numpy` (but our data is now on our GPU and doesn't need to be communicated over):

In [9]:
import cupy as cp
x, y = cp.random.uniform(low=-1, high=1, size=(2, n_runs))

...And check to see if if these random coordinates are in our circle or not using identical notation

In [10]:
result = x ** 2 + y ** 2 <= 1 # on GPU
n_in_circle = cp.sum(result) # on GPU
pi = 4 * n_in_circle / n_runs # on GPU
pi.get() # explicitly send estimated value back to our CPU

array(3.14150016)

CuPy also conveniently includes a benchmarking function that explicitly times how much work is being done on GPU versus CPU. We can see, for instance, that if we run the same code and `.get()` or result back on our CPU, there is a significant amount of time that is spent on CPU operations (reinforcing the need to minimize data transfer between CPU and GPU when possible!):

In [11]:
from cupyx.profiler import benchmark

def cupy_pi_est(x, y):
  # Compute on GPU via CuPy arrays: ~173k us (microseconds)
  result = x ** 2 + y ** 2 <= 1
  n_in_circle = cp.sum(result)
  pi = 4 * n_in_circle / n_runs

  # Get pi back to host CPU from GPU: ~173k us (microseconds)
  # half of time is spent just getting pi back to host CPU from GPU device
  pi_cpu = pi.get()
  
  return pi_cpu

# Similar to numba performance without storing intermediate results on GPU ~300 ms
benchmark(cupy_pi_est, (x, y), n_repeat=10)

cupy_pi_est         :    CPU:175934.926 us   +/-1286.304 (min:173277.126 / max:177631.907) us     GPU-0:175965.796 us   +/-1286.216 (min:173310.913 / max:177664.322) us

Whereas if we perform the same operation without bringing the estimated value back to the CPU via `.get()`, our GPU time remains the same, but there is quite a bit less time spent overall (since there is no data transfer time):

In [12]:
def cupy_pi_est(x, y):
  # Compute on GPU via CuPy arrays: ~173k us (microseconds)
  result = x ** 2 + y ** 2 <= 1
  n_in_circle = cp.sum(result)
  pi = 4 * n_in_circle / n_runs

  # don't send result back to CPU here for timing purposes
  
  return pi

# Similar to numba performance without storing intermediate results on GPU ~300 ms
benchmark(cupy_pi_est, (x, y), n_repeat=10)

cupy_pi_est         :    CPU:  615.064 us   +/-56.303 (min:  543.270 / max:  748.027) us     GPU-0:177046.494 us   +/-922.323 (min:175527.969 / max:178503.677) us